### 疫情地图

In [1]:
import json
from datetime import date, datetime
import datetime
import requests

import numpy as np 
import pandas as pd
import pyecharts
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode

In [2]:
# 生成更新日期
update_date = date.today()
# update_date = '2020-04-25'

In [3]:
# 加载数据
area_df = pd.read_csv('./api_datas/clean/area_clean_{}.csv'.format(update_date), encoding='utf-8')
area_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 19 columns):
cities                   257 non-null object
comment                  257 non-null object
confirmedCount           257 non-null int64
continentEnglishName     257 non-null object
continentName            257 non-null object
countryEnglishName       257 non-null object
countryFullName          218 non-null object
countryName              257 non-null object
curedCount               257 non-null int64
currentConfirmedCount    257 non-null int64
deadCount                257 non-null int64
locationId               257 non-null int64
provinceEnglishName      257 non-null object
provinceName             257 non-null object
provinceShortName        257 non-null object
suspectedCount           257 non-null int64
updateTime               257 non-null int64
deadRate                 256 non-null float64
curedRate                256 non-null float64
dtypes: float64(2), int64(7), object(10)
memo

#### 对大洲进行分组，统计各个洲的新冠累计确诊数量

In [4]:
confirmed_per_continent = area_df.groupby('continentName')[['confirmedCount', 'currentConfirmedCount']].agg(['sum']).reset_index()
confirmed_per_continent.columns = ["continentName", "total_counts", "current_counts"]
# 数量降序排列
confirmed_per_continent = confirmed_per_continent.sort_values(by='total_counts', ascending=False)
confirmed_per_continent

,continentName,total_counts,current_counts
5,欧洲,1728151,823043
2,北美洲,1692244,1221210
0,亚洲,921166,331831
3,南美洲,487157,288812
6,非洲,92078,54785
4,大洋洲,8495,610
1,其他,733,146


#### 各大洲新冠累计确诊人数：柱状图

In [5]:
bar = (Bar()    # 创建图形对象
       #添加坐标轴数据
       .add_xaxis(confirmed_per_continent['continentName'].values.tolist())
       .add_yaxis("人数", confirmed_per_continent['total_counts'].values.tolist())
       # 配置系列数据项：调整Bar图上的数据标签的字体
       .set_series_opts(label_opts=opts.LabelOpts(font_size=10))
       # 配资全局数据项：设置主标题、副标题，以及x轴的标签字体
       .set_global_opts(title_opts=opts.TitleOpts(title="各大洲新冠累计确诊人数", subtitle="单位：人"), 
                        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=0, font_size=12)) )
      )

# 渲染图形，在notebook中显示
bar.render_notebook()

 - 结论：从以上柱状图可以看出，新冠确诊病例最多的大洲分别是欧洲，北美洲，亚洲。确诊数远远高于其他的大洲。

#### 各大洲新冠当前（存量）确诊人数：柱状图

In [6]:
bar = (Bar()    # 创建图形对象
       #添加坐标轴数据
       .add_xaxis(confirmed_per_continent['continentName'].values.tolist())
       .add_yaxis("人数", confirmed_per_continent['current_counts'].values.tolist())
       # 配置系列数据项：调整Bar图上的数据标签的字体
       .set_series_opts(label_opts=opts.LabelOpts(font_size=10))
       # 配资全局数据项：设置主标题、副标题，以及x轴的标签字体
       .set_global_opts(title_opts=opts.TitleOpts(title="各大洲新冠当前存量确诊人数", subtitle="单位：人"), 
                        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=0, font_size=12)) )
      )

# 渲染图形，在notebook中显示
bar.render_notebook()

 - 从以上图表可以看出，当前新冠确诊人数存量最多的是北美洲，其次是欧洲和亚洲。

#### 国内疫情地图

In [7]:
# 提取国内各省份的新冠确诊数据
china_province_data = area_df[area_df['countryName']=='中国'][['provinceShortName', 'confirmedCount']]
china_province_data

,provinceShortName,confirmedCount
137,中国,84505
147,内蒙古,217
148,吉林,152
184,澳门,45
185,台湾,440
186,上海,666
187,湖北,68135
188,陕西,308
189,香港,1055
190,河北,328


In [9]:
# 删除第一条总计数据
china_province_data.drop(labels=china_province_data.index[0], inplace=True)
china_province_data

,provinceShortName,confirmedCount
147,内蒙古,217
148,吉林,152
184,澳门,45
185,台湾,440
186,上海,666
187,湖北,68135
188,陕西,308
189,香港,1055
190,河北,328
191,北京,593


In [10]:
# 将国内各个省份的确诊数据转换为 list
province_data = []
for idx, row in china_province_data.iterrows():
    province_data.append((row.values[0], row.values[1]))
    
print(province_data)

[('内蒙古', 217), ('吉林', 152), ('澳门', 45), ('台湾', 440), ('上海', 666), ('湖北', 68135), ('陕西', 308), ('香港', 1055), ('河北', 328), ('北京', 593), ('天津', 192), ('广东', 1590), ('黑龙江', 945), ('山东', 788), ('海南', 169), ('福建', 356), ('辽宁', 149), ('山西', 198), ('江苏', 653), ('云南', 185), ('浙江', 1268), ('安徽', 991), ('四川', 561), ('重庆', 579), ('贵州', 147), ('湖南', 1019), ('甘肃', 139), ('河南', 1276), ('江西', 937), ('广西', 254), ('宁夏', 75), ('新疆', 76), ('青海', 18), ('西藏', 1)]


In [11]:
# province_data = []
# for item in data['results']:
#     if item['countryName'] == '中国':
#         province_data.append((item['provinceShortName'], item['confirmedCount']))

china_map = (
        Map(init_opts=opts.InitOpts(theme='dark', width='400'))
        .add("累计确诊人数", province_data, "china", is_map_symbol_show=False, is_roam=False)
        .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
        .set_global_opts(title_opts=opts.TitleOpts(title="新型冠状病毒全国疫情地图", 
                                                   subtitle="更新日期：{}".format(update_date)), 
                         legend_opts=opts.LegendOpts(is_show=False), 
                         visualmap_opts=opts.VisualMapOpts(is_show=True, max_=1000, is_piecewise=False, 
                                                           range_color=['#FFFFE0', '#FFA07A', '#CD5C5C', '#8B0000']))
)

china_map.render_notebook()

 - 结论：从上面的疫情地图可以看出，颜色越深的区域，表示累计确诊病例越多，即疫情越严重。截止目前，疫情较严重的地区主要是东北，河南，安徽，湖北，湖南，江西，浙江，广东等省份。

#### 中国疫情热力图

In [12]:
china_cities_data = area_df[area_df['countryName']=='中国']['cities']
china_cities_data

137                                                   暂无
147    [{'cityName': '境外输入人员', 'currentConfirmedCount...
148    [{'cityName': '吉林市', 'currentConfirmedCount': ...
184                                                   暂无
185                                                   暂无
186    [{'cityName': '境外输入', 'currentConfirmedCount':...
187    [{'cityName': '武汉', 'currentConfirmedCount': 7...
188    [{'cityName': '境外输入', 'currentConfirmedCount':...
189                                                   暂无
190    [{'cityName': '唐山', 'currentConfirmedCount': 0...
191    [{'cityName': '朝阳区', 'currentConfirmedCount': ...
201    [{'cityName': '境外输入', 'currentConfirmedCount':...
202    [{'cityName': '广州', 'currentConfirmedCount': 2...
203    [{'cityName': '境外输入', 'currentConfirmedCount':...
204    [{'cityName': '境外输入', 'currentConfirmedCount':...
205    [{'cityName': '三亚', 'currentConfirmedCount': 1...
206    [{'cityName': '福州', 'currentConfirmedCount': 0...
207    [{'cityName': '沈阳', 'cur

In [13]:
import re

cities_data = []
def match_func(s):
    reg = re.compile(r"{\'cityName\': \'([\u4e00-\u9fa5]+)\', \'currentConfirmedCount\': (\d+)")
    res = re.findall(reg, s)
    return res

for c in china_cities_data.values.tolist():
    if c == '暂无':
        continue
    else: 
        match_res = match_func(c)
        cities_data.extend(match_res)

cities_data

[('境外输入人员', '19'),
 ('呼伦贝尔', '1'),
 ('鄂尔多斯', '0'),
 ('包头', '0'),
 ('赤峰', '0'),
 ('锡林郭勒盟', '0'),
 ('巴彦淖尔', '0'),
 ('呼和浩特', '0'),
 ('通辽', '0'),
 ('乌兰察布', '0'),
 ('乌海市', '0'),
 ('兴安盟', '0'),
 ('吉林市', '25'),
 ('长春', '0'),
 ('四平市', '0'),
 ('辽源', '0'),
 ('延边', '0'),
 ('公主岭', '0'),
 ('通化', '0'),
 ('松原', '0'),
 ('梅河口', '0'),
 ('白城', '0'),
 ('境外输入', '16'),
 ('外地来沪', '1'),
 ('浦东新区', '0'),
 ('宝山区', '0'),
 ('闵行区', '0'),
 ('徐汇区', '0'),
 ('静安区', '0'),
 ('松江区', '0'),
 ('长宁区', '0'),
 ('普陀区', '0'),
 ('杨浦区', '0'),
 ('嘉定区', '0'),
 ('奉贤区', '0'),
 ('虹口区', '0'),
 ('黄浦区', '0'),
 ('青浦区', '0'),
 ('金山区', '0'),
 ('崇明区', '0'),
 ('武汉', '7'),
 ('孝感', '0'),
 ('黄冈', '0'),
 ('荆州', '0'),
 ('鄂州', '0'),
 ('随州', '0'),
 ('襄阳', '0'),
 ('黄石', '0'),
 ('宜昌', '0'),
 ('荆门', '0'),
 ('咸宁', '0'),
 ('十堰', '0'),
 ('仙桃', '0'),
 ('天门', '0'),
 ('恩施州', '0'),
 ('潜江', '0'),
 ('神农架林区', '0'),
 ('境外输入', '2'),
 ('西安', '0'),
 ('安康', '0'),
 ('汉中', '0'),
 ('咸阳', '0'),
 ('渭南', '0'),
 ('宝鸡', '0'),
 ('延安', '0'),
 ('铜川', '0'),
 ('商洛', '0'),
 ('榆林', '

In [15]:
## 绘制疫情热力图
geo = (
        Geo(init_opts=opts.InitOpts(theme='dark', width='400'))
        .add_schema(maptype="china", zoom=3, center=[114.31,30.52])
        .add("现存确诊人数", 
             [i for i in cities_data if i[0] in pyecharts.datasets.COORDINATES.keys()],
             type_='heatmap', symbol_size=3, progressive=50)
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(title_opts=opts.TitleOpts(title="新型冠状肺炎新增病例热力图", 
                                                   subtitle="更新日期：{}".format(update_date), 
                                                   pos_left='right'),
                         legend_opts=opts.LegendOpts(is_show=False),
                         visualmap_opts=opts.VisualMapOpts(is_show=True, 
                                                           is_piecewise=False, 
                                                           range_color=['blue', 'green', 'yellow', 'orange', 'red'])
                        )
)

geo.render_notebook()

 - 从地图可以看出，新冠现存确诊人数主要分布在 黑龙江，河北，湖北，广东一带。与之前大爆发时期相比，有了很大的好转。

### 国内疫情趋势
以下Tableau可视化展示了新冠疫情在国内的基本情况。（截止2020-04-30）

#### 每日新增
<img src='imgs/国内每日新增数据.jpg' align='center'>

#### 累计数据
<img src='imgs/国内新冠累计数据.jpg' align='center'>

#### 治愈率与死亡率曲线
<img src='imgs/国内新冠治愈率和死亡率曲线.jpg' align='center'>

### 趋势预测
    以下Tableau可视化，预测了未来30日国内新冠疫情的基本走势（仅供参考）。

#### 每日新增预测
<img src='imgs/预测：未来30天国内每日新增数据.jpg' align='center'>

#### 累计确诊预测
<img src='imgs/预测：未来30天国内新冠累计数据.jpg' align='center'>

#### 治愈率及死亡率预测
<img src='imgs/预测：未来30天治愈率和死亡率.jpg' align='center'>

### 海外疫情趋势
    今年3月份以来，新冠疫情在海外呈现出爆发增长的趋势。以下展现了在Tableau中对世界新冠疫情的可视化。（截止2020-04-25）

#### 治愈率
<img src='imgs/世界各国治愈率（地图）.jpg' align='center'>

#### 死亡率
<img src='imgs/世界各国死亡率（地图）.jpg' align='center'>

#### 治愈率和死亡率对比
<img src='imgs/主要国家治愈率和死亡率（多维柱状图）.jpg' align='center'>

#### 累计确诊
<img src='imgs/世界主要国家累计确诊数据（条形图）.jpg' align='center'>

#### 现存确诊
<img src='imgs/世界主要国家现存确诊数据（地图）.jpg' align='center'>

### 根据网址 [疫情每日消息](https://lab.isaaclin.cn/nCoV/api/news) 获取到的新闻数据，制作的词云图：
<img src='imgs/Cov19_WordCloud.jpg' align='center'>
